## Initial aggregation notebook to create the gold table

In [0]:
import os

os.environ['BUCKET_NAME'] = 'databricks-745bwkyiddeq9fthttjahg-cloud-storage-bucket'
bucket_name = os.getenv('BUCKET_NAME')
print(bucket_name)

In [0]:
from pyspark.sql.functions import col, sum as sum_, avg

silver_path = f"s3://{bucket_name}/ohio-prod/3903799048317088/mnt/silver/daily_activity_clean"
df_silver = spark.read.format("delta").load(silver_path)

df_gold_daily_steps = (df_silver
    .groupBy("Id", "ActivityDate")
    .agg(
        sum_("TotalSteps").alias("total_steps"),
        sum_("Calories").alias("total_calories"),
        avg("TotalDistance").alias("avg_distance") 
    )
)

gold_path = f"s3://{bucket_name}/ohio-prod/3903799048317088/mnt/gold/daily_activity_summary"

(df_gold_daily_steps.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .save(gold_path))

print("Gold Table 1 (daily_activity_summary) created successfully!")

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS fitness_gold
MANAGED LOCATION 's3://databricks-745bwkyiddeq9fthttjahg-cloud-storage-bucket/ohio-prod/3903799048317088/mnt/gold/';

In [0]:
%sql
CREATE TABLE IF NOT EXISTS fitness_gold.daily_activity_summary
LOCATION 's3://databricks-745bwkyiddeq9fthttjahg-cloud-storage-bucket/ohio-prod/3903799048317088/mnt/gold/daily_activity_summary/';

In [0]:
%sql
DESCRIBE EXTENDED fitness_gold.daily_activity_summary;

In [0]:
%sql
SELECT * FROM fitness_gold.daily_activity_summary;